# План работы:
- Кроме Videos.md должен быть файл Videos.html - а лучше ipynb - тогда будет встраивание в обоих формах видео, описание в предыдущей markdown-ячейке.
- Для оценки структуры курса и генерации элементов - ячейка с считыванием всех имеющихся блоков, в которых source для Markdown содержит знаки решетки. Это автоматически отправлять в Syllabus
- Сделать взаимодополняющимися разделы "Литература" - Переносить из файла Literature.md в соответствующие разделы и обратно

In [281]:
import os
import json
os.chdir('../course') # Все действия осуществляются из директории курса

from IPython.display import display, Markdown, Latex

# Определение путей всех имеющихся в курсе ipynb-файлов

In [282]:
ipynb_paths = []

for dirname, dirnames, filenames in os.walk(os.getcwd()):
    for filename in filenames:
        if filename.endswith('.ipynb') & (not filename.endswith('-checkpoint.ipynb')):
            ipynb_paths.append(os.path.join(dirname, filename))

ipynb_paths = [ipynb_path for ipynb_path in ipynb_paths if 'Videos' not in ipynb_path]

# Актуальное содержание в READ.me файле

In [283]:
def chapters():
    chs = dict()
    for i, ipynb_path in enumerate(sorted(ipynb_paths)):
        with open(ipynb_path) as f:
            data = json.load(f)
            name = data['cells'][0]['source'][0]  # Заголовок файла это всегда первая ячейка
            markdown_path = os.path.join(os.path.basename(os.path.dirname(ipynb_path)), os.path.basename(ipynb_path))
            # срез так как название начинается с решетки и пробела
            chs[ipynb_path] = '{0}. [{1}]({2})'.format(i+1, name[2:].strip(), markdown_path)
    return chs
            
with open('README.md') as f:
    blocks = f.read().split('#')
    for i, block in enumerate(blocks):
        if block.startswith(' Содержание курса'):
            blocks[i] = ' Содержание курса\n\n' + '\n'.join([chapters()[key] for key in sorted(chapters())]) + '\n\n'
    f.close()

with open('README.md', 'w') as f:           
    f.write("#".join(blocks)) 
    f.close()

# print('Ниже идет текущий вид сгенерированного файла READ.me')
# with open('README.md') as f:
#     display(Markdown(f.read()))
#     f.close()

# Полная структура курса

In [284]:
# Рассматриваем только для markdown cells

# Генерация страницы с полным списком видео

Для удобства просмотра видео на досуге сделаем общую страницу, в которой видеофрагменты из всех Jupyter-блокнотов будут собираться в общий файл [Videos.md]().

In [ ]:
import IPython.nbformat as nbf
from nbconvert.preprocessors import ExecutePreprocessor
ep = ExecutePreprocessor(timeout=600, kernel_name='python3')

md_first_lines = '''
# Видеотека

Данный файл содержит подборку из всех видеофрагментов,
представленных в различных частях курса "Наноматериалы".

Приведенные видеофрагменты не являются обязательными
к просмотру, но позволяют увидеть рассматриваемые проблемы
с позиций других исследователей. Некоторые видеоролики
полезны как примеры лабораторных практик по синтезу
и диагностике различных форм наноматериалов.
'''

videos_path = 'Videos/Videos.ipynb'

with open(videos_path, 'w') as videos_file:
    cells = []
    cells.append(nbf.v4.new_markdown_cell(source=md_first_lines))
    cells.append(nbf.v4.new_code_cell(source='from IPython.lib.display import YouTubeVideo'))
    for ipynb_path in sorted(chapters().keys()):
        with open(ipynb_path) as f:
            head = '# {}'.format(chapters()[ipynb_path])
            cells.append(nbf.v4.new_markdown_cell(source=head))
            data = json.load(f)
            for cell in data['cells']:
                if (cell['cell_type'] == 'code') & ('YouTubeVideo' in ''.join(cell['source'])):
                    d = "".join(cell['source'])
                    d = d.replace('\n', ' ')
                    try:
                        d = d[3:d[d.index("'''")+3:].index("'''")+3]
                        d = d.lstrip()
                    except ValueError:
                        d = '**У этого видео пока нет описания.**'
                    for line in cell['source']:
                        if "YouTubeVideo('" in line.strip():
                            short_link = line[len("YouTubeVideo('"):-len(")'")]
                    v = 'YouTubeVideo("{}")'.format(short_link)
                    cells.append(nbf.v4.new_markdown_cell(source=d))
                    cells.append(nbf.v4.new_code_cell(source=v))
    nb = nbf.v4.new_notebook(cells=cells)
    nbf.write(nb, videos_file)
    os.system('jupyter nbconvert --execute {}'.format(videos_path))

# Общий объем текстовой части

In [ ]:
total_symbols = 0
for ipynb_path in sorted(chapters().keys()):
    with open(ipynb_path) as f:
        data = json.load(f)
        total_symbols += sum([len(''.join(cell['source'])) for cell in data['cells'] if cell['cell_type'] == 'markdown'])

print('Общий объем непрограммного текста - {} символов, что соответствует примерно {} листов А4.'.format(total_symbols, total_symbols//1800))